In [1]:
import numpy as np
import pandas as pd
import pickle
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from linearmodels.panel import compare

import FixedEffectModelPyHDFE.api as FEM
from fixedeffect.fe import fixedeffect, did, getfe

from fixedeffect.fe import fixedeffect # for 3-way fixed https://pypi.org/project/FixedEffectModel/
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib

import seaborn as sns

matplotlib.rcParams['font.family'] = 'serif'

In [2]:
data_dir = '/home/simon/Documents/Bodies/data/done_dfs/'

with open(f'{data_dir}bodies_df_2022_v1_4.pkl', 'rb') as file:
    bodies_df = pickle.load(file)

In [3]:
# if you only want more robust observatino:

bodies_df = bodies_df[(bodies_df['distance_days'] <= 48) | (bodies_df['location_annot'] == 1)].copy() # only locations you are certain of
bodies_df = bodies_df[bodies_df['city'] != 'nan']

bodies_df = bodies_df[bodies_df['year'] >= 2003]
lwd = [294, 299, 300] # likely wrong dates/months. 
bodies_df = bodies_df[~bodies_df['month_id'].isin(lwd)].copy()

# indtil det er styr på det her:
bodies_df = bodies_df[bodies_df['publication'] != 'Published']

In [4]:
#bodies_df[['Published', 'Raw', 'Submitted']] = pd.get_dummies(bodies_df[['publication']])
bodies_df[['Raw', 'Submitted']] = pd.get_dummies(bodies_df[['publication']])

bodies_df.drop(columns= 'publication', inplace= True)

# for FE
bodies_df.set_index(['gid', 'month_id'], inplace = True)

In [5]:
bodies_df = bodies_df[bodies_df['person_mean']>0]

bodies_df['lgarments'] = np.log(bodies_df['religiousGarmentFemale_mean'] + 1)
bodies_df['lfemale'] = np.log(bodies_df['female_mean'] + 1)

bodies_df['female_dummy'] = (bodies_df['female_mean'] > 0)*1
bodies_df['garment_dummy'] = (bodies_df['religiousGarmentFemale_mean'] > 0)*1

bodies_df['pp'] = bodies_df['public_mean'] - bodies_df['privat_mean']
bodies_df['log_best_public'] = bodies_df['log_best'] * bodies_df['pp']


# public_df = bodies_df[(bodies_df['pp']>0)&(bodies_df['person_mean']>0)]
# privat_df = bodies_df[(bodies_df['pp']<0)&(bodies_df['person_mean']>0)]

public_df = bodies_df[bodies_df['pp']>0]
privat_df = bodies_df[bodies_df['pp']<0]